In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from scripts.saver import save_dataset, save_plot
from scripts.eda_plots import plot_numeric_boxplots
from scripts.data_loader import load_raw
from scripts.preprocessing import (rename_columns,
                                  encode_features,
                                   scale_features)


import warnings
warnings.filterwarnings("ignore")

# Load the dataset
df = load_raw("data-cleaned.csv")
# Redefine X and y after encoding
X = df.drop(columns=["final_grade", "passed", "student_id"])

# Create classification target: pass if final_grade >= 10
df["passed"] = (df["final_grade"] >= 10).astype(int)

# Drop non-feature columns
X = df.drop(columns=["final_grade", "passed", "student_id"])
y_reg = df["final_grade"]
y_clf = df["passed"]

# Train-test split
X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, random_state=42
)

# Regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_reg_train)
y_reg_pred = reg_model.predict(X_test)
reg_r2 = r2_score(y_reg_test, y_reg_pred)

# # Classification model
# clf_model = LogisticRegression(max_iter=1000)
# clf_model.fit(X_train, y_clf_train)
# y_clf_pred = clf_model.predict(X_test)
# clf_acc = accuracy_score(y_clf_test, y_clf_pred)

# Report results
print(f"📈 Regression R² Score: {reg_r2:.3f}")
# print(f"🎯 Classification Accuracy: {clf_acc:.3f}")


KeyError: "['passed'] not found in axis"

In [10]:
df = load_raw("data-cleaned.csv")
print(df.dtypes)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score

# Drop problematic non-numeric column
df = df.drop(columns=["school"])

# Create binary classification target
df["passed"] = (df["final_grade"] >= 10).astype(int)

# Define features and targets
X = df.drop(columns=["final_grade", "passed", "student_id"])
y_reg = df["final_grade"]
y_clf = df["passed"]

# Train/test split
X_train, X_test, y_reg_train, y_reg_test, y_clf_train, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, random_state=42
)

# Regression
reg_model = LinearRegression()
reg_model.fit(X_train, y_reg_train)
y_reg_pred = reg_model.predict(X_test)
reg_r2 = r2_score(y_reg_test, y_reg_pred)

# Classification
clf_model = LogisticRegression(max_iter=1000)
clf_model.fit(X_train, y_clf_train)
y_clf_pred = clf_model.predict(X_test)
clf_acc = accuracy_score(y_clf_test, y_clf_pred)

# Results
print(f"📈 Regression R² Score: {reg_r2:.3f}")
print(f"🎯 Classification Accuracy: {clf_acc:.3f}")


student_id                           int64
school                              object
sex                                float64
age                                float64
address                            float64
family_size                        float64
parent_status                      float64
mother_education                   float64
father_education                   float64
travel_time                        float64
study_time                         float64
past_failures                      float64
school_support                     float64
family_support                     float64
extra_paid_classes                 float64
activities                         float64
attended_nursery                   float64
wants_higher_edu                   float64
internet_access                    float64
in_romantic_relationship           float64
family_relationship_quality        float64
free_time                          float64
social_life                        float64
weekday_alc

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [11]:
print("🔍 NaN values per column:")
print(X.isnull().sum().sort_values(ascending=False).head())

print("\n🧪 Any infinite values?", np.isinf(X.values).any())


🔍 NaN values per column:
weekend_alcohol_use    130
sex                      0
free_time                0
weekday_alcohol_use      0
health_status            0
dtype: int64

🧪 Any infinite values? False
